Les importations de modules :

In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat
import scipy
import numpy as np
from scipy.special import kv, iv


Les fonctions utilisées sont :

In [ ]:

#ouvrir un fichier et return une liste avec les valeurs

#-----------
def ouverture_fichier(nomfichier):
    with open(nomfichier, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        xc = []
        i = 0
        for row in csvreader:
            xc.append(float(row[1]))
            i += 1
            if i >= 1000:
                break
    return xc
#-----------


#Equations

#------------ Calcul de delta z

# Paramètres du circuit
omega = 0                       # Fréquence angulaire en rad/s
mu_0 = 0                        # Perméabilité du vide en Tm/A
mu = 0                          # Permeabilite du materiaux
mu_m = mu / mu_0                # µ/µ_0
a = 0                           # Rayon du solénoïde en m
n = 0                           # Nombre de spires du solénoïde
epsilon_0 = 0                   # Permittivité du vide en F/m
epsilon = 0                     # Permittivité de la tige en F/m
b = 0                           # Rayon de la tige conductrice en m
rho = 0                         # Résistivité de la tige conductrice en ohm/m

# Calculs supplémentaires
zeta = np.sqrt((omega ** 2) * mu_0 * epsilon_0)
eta = np.sqrt(zeta ** 2 - (omega ** 2) * mu_0 * epsilon_0)
gamma = b * np.sqrt(zeta ** 2 + (1j * omega * mu_m / rho) - (omega ** 2) * mu_m * epsilon)

# Fonctions de Bessels modifiées
I_0 = iv(0, gamma)
I_1 = iv(1, gamma)
K_0 = kv(0, eta * a)
K_1 = kv(1, eta * a)


def Calcul_integrale() :

    return

integrale = Calcul_integrale()

def delta_z() :

    resultat = ... * integrale

    return resultat

#--------------



Generation de Graphiques :

In [ ]:

#Recevoir/ouvrir CSV thermometre
nom_fichier_temperature = "/Users/ewenmaugey/Desktop/PHS2903/codeprojet/resistance_ewen.lvm"     # Remplacez "mon_fichier.csv" par le nom de votre fichier CSV
                                
nom_fichier_resistivite = "/Users/ewenmaugey/Desktop/PHS2903/codeprojet/resistance_ewen.lvm"     # Remplacez "mon_fichier.csv" par le nom de votre fichier CSV

#Faire les variables x = resistivite | y = temperature

x = ouverture_fichier(nom_fichier_temperature)
print('temperature',x)  

y = ouverture_fichier(nom_fichier_resistivite)
print('resistivite',y)  

# Tracer le graphique
plt.plot(x, y)  
plt.xlabel('Température')  
plt.ylabel('Résistivité')  
plt.title('Graphique de la résistivite de l\'echantillon en fonction de la temperature')  
plt.grid(True)  
plt.show()  


Communication entre Ordinateur-Arduino

In [ ]:
import serial
import csv
import time

# Ouvre la connexion série avec la carte Arduino
arduino = serial.Serial('/dev/ttyACM0', 9600)  # Assure-toi de changer '/dev/ttyACM0' par le bon port série

# Envoie une commande à l'Arduino pour qu'il commence l'acquisition de données
arduino.write(b'start_data_acquisition\n')

# Attend la réponse de l'Arduino
while True:
    response = arduino.readline().strip()
    if response == b'data_ready':
        break

In [ ]:
#C++

Communication entre Ordianteur-NimyDaq

In [ ]:
#py

In [ ]:
#Labview